Introduction:
The electricity market operates in a dynamic environment where supply and demand continually fluctuate, leading to varying electricity prices. In this context, the Hourly Ontario Energy Price (HOEP) dataset becomes a valuable resource, providing insights into the hourly pricing of electricity in Ontario, Canada. This dataset, administered by the Independent Electricity System Operator (IESO), captures the intricacies of electricity power exchanges, influencing decisions across the energy sector.

Electricity Power Exchanges and the IESO:
Electricity power exchanges represent a crucial aspect of the energy landscape, facilitating the buying and selling of electricity to meet the ever-changing demand. The IESO, as the Independent Electricity System Operator for Ontario, plays a central role in maintaining the reliability and efficiency of the electric grid. It oversees the electricity market, ensuring a balance between supply and demand, and administers the HOEP, reflecting the market price for electricity on an hourly basis.

Necessity of Time Series Modeling for Electricity Price Forecasting:
In the realm of electricity markets, time series modeling, specifically forecasting, emerges as a vital tool for energy decision-makers. By predicting future electricity prices, decision-makers can effectively manage the balance between supply and demand throughout the day. This proactive approach helps control sporadic price collapses, reduces the occurrence of negative prices, and contributes to the overall reliability of the electric grid.

Electricity generators, as suppliers in the market, can leverage price forecasts to optimize their offering strategies. Understanding the projected price trends in subsequent hours empowers generators to make informed decisions, enhancing their overall market performance. On the demand side, energy traders utilize price projections to identify lucrative opportunities. This strategic insight allows them to navigate the market effectively, maximizing their gains.

Specifically, certain market participants, such as dispatchable loads, rely on price forecasts to inform their bidding strategies in auctions. Dispatchable loads, representing consumers with adjustable electricity consumption, play a pivotal role in the market's dynamics. Their ability to anticipate future price trends, influenced by accurate forecasting, allows them to make strategic decisions in bidding strategies.

Objective:
In light of the factors mentioned above, our objective is to develop a time series model for electricity price forecasting using the HOEP dataset. By analyzing historical pricing patterns, we aim to create a robust model that can provide accurate forecasts, benefiting both suppliers and consumers in the Ontario electricity market. The ultimate goal is to contribute to the effective management of the electricity grid, promote market efficiency, and support decision-makers in making informed choices based on reliable price projections.








1) Loading data

In [476]:
import pandas as pd
import matplotlib.pyplot as plt

In [454]:
#create a file path 
file_path=['time series data/PUB_PriceHOEPPredispOR_2018.csv',
           'time series data/PUB_PriceHOEPPredispOR_2019.csv',
          'time series data/PUB_PriceHOEPPredispOR_2020.csv',
          'time series data/PUB_PriceHOEPPredispOR_2021.csv',
          'time series data/PUB_PriceHOEPPredispOR_2022.csv',
          'time series data/PUB_PriceHOEPPredispOR_2023.csv']
file_path[0][-9:-1]

'_2018.cs'

In [455]:
# Crating a dictionary to importing dataframes of each year
HOEP_data={}
#Load data into dictionary
for i in range(len(file_path)):
    HOEP_data[file_path[i][-8:]]=pd.read_csv(file_path[i])

2) Data preprocessing


In [457]:
HOEP_data
# Deleting first three rows of each dataframe
# changing first row into dataframe label
# updating index
for key,value in HOEP_data.items():
    HOEP_data[key]= HOEP_data[key].iloc[2:]
    HOEP_data[key].columns=HOEP_data[key].iloc[0]
    HOEP_data[key]= HOEP_data[key].iloc[1:]
    HOEP_data[key].reset_index(drop=True, inplace=True)
    
for key,value in HOEP_data.items():
    print(f'key:\n{key}\n\n value:\n{value}')

key:
2018.csv

 value:
2           Date Hour   HOEP Hour 1 Predispatch Hour 2 Predispatch  \
0     2018-01-01    1  51.29              40.62              35.50   
1     2018-01-01    2  43.59              40.93              40.57   
2     2018-01-01    3  93.60              49.84              45.30   
3     2018-01-01    4  54.78              55.60              40.04   
4     2018-01-01    5  14.35              40.04              34.80   
...          ...  ...    ...                ...                ...   
8755  2018-12-31   20   5.62               5.92               8.80   
8756  2018-12-31   21   8.95               5.83               5.96   
8757  2018-12-31   22   5.81               6.00               5.57   
8758  2018-12-31   23   2.87               5.86               5.92   
8759  2018-12-31   24   0.00               0.00               0.00   

2    Hour 3 Predispatch OR 10 Min Sync OR 10 Min non-sync OR 30 Min  
0                 40.32           0.20               0.20      0.1

In [458]:
# cheking for missing value:
for key, value in HOEP_data.items():
    print(f'\n Year {key} number of missing value: \n {value.isna().sum()}')


 Year 2018.csv number of missing value: 
 2
Date                   0
Hour                   0
HOEP                   0
Hour 1 Predispatch    27
Hour 2 Predispatch    27
Hour 3 Predispatch    27
OR 10 Min Sync         0
OR 10 Min non-sync     0
OR 30 Min              0
dtype: int64

 Year 2019.csv number of missing value: 
 2
Date                   0
Hour                   0
HOEP                   0
Hour 1 Predispatch    21
Hour 2 Predispatch    21
Hour 3 Predispatch    21
OR 10 Min Sync         0
OR 10 Min non-sync     0
OR 30 Min              0
dtype: int64

 Year 2020.csv number of missing value: 
 2
Date                   0
Hour                   0
HOEP                   0
Hour 1 Predispatch    19
Hour 2 Predispatch    19
Hour 3 Predispatch    19
OR 10 Min Sync         0
OR 10 Min non-sync     0
OR 30 Min              0
dtype: int64

 Year 2021.csv number of missing value: 
 2
Date                   0
Hour                   0
HOEP                   0
Hour 1 Predispatch    16
Hour 2

In [459]:
#Drop extra columns
extra_columns=['Hour 1 Predispatch',
               'Hour 2 Predispatch',
               'Hour 3 Predispatch',
               'OR 10 Min Sync',
               'OR 10 Min non-sync','OR 30 Min']
HOEP_data_clean={}
for key in HOEP_data:
    HOEP_data_clean[key]=HOEP_data[key].drop(columns=extra_columns)
    
#print the modified data frame:
for key,value in HOEP_data_clean.items():
    print(f'{key}:\n{value}')

2018.csv:
2           Date Hour   HOEP
0     2018-01-01    1  51.29
1     2018-01-01    2  43.59
2     2018-01-01    3  93.60
3     2018-01-01    4  54.78
4     2018-01-01    5  14.35
...          ...  ...    ...
8755  2018-12-31   20   5.62
8756  2018-12-31   21   8.95
8757  2018-12-31   22   5.81
8758  2018-12-31   23   2.87
8759  2018-12-31   24   0.00

[8760 rows x 3 columns]
2019.csv:
2           Date Hour   HOEP
0     2019-01-01    1   0.00
1     2019-01-01    2   0.00
2     2019-01-01    3  -0.11
3     2019-01-01    4  -1.84
4     2019-01-01    5  -0.47
...          ...  ...    ...
8755  2019-12-31   20   0.00
8756  2019-12-31   21   0.00
8757  2019-12-31   22  -0.05
8758  2019-12-31   23  -0.04
8759  2019-12-31   24  -0.02

[8760 rows x 3 columns]
2020.csv:
2           Date Hour   HOEP
0     2020-01-01    1   0.00
1     2020-01-01    2   0.00
2     2020-01-01    3   0.00
3     2020-01-01    4   0.00
4     2020-01-01    5   0.00
...          ...  ...    ...
8779  2020-12-31   20

In [460]:
# checking for duplicates row in each dataframe
for key,value in HOEP_data_clean.items():
    print(f'Year: {key} number of duplicate rows: {value.duplicated().sum()}')

Year: 2018.csv number of duplicate rows: 0
Year: 2019.csv number of duplicate rows: 0
Year: 2020.csv number of duplicate rows: 0
Year: 2021.csv number of duplicate rows: 0
Year: 2022.csv number of duplicate rows: 0
Year: 2023.csv number of duplicate rows: 0


In [461]:
#cheking for datatypes of columns of each year:
for key,value in HOEP_data_clean.items():
    print(f' Dataframe:{key}\n{value.dtypes}')

 Dataframe:2018.csv
2
Date    object
Hour    object
HOEP    object
dtype: object
 Dataframe:2019.csv
2
Date    object
Hour    object
HOEP    object
dtype: object
 Dataframe:2020.csv
2
Date    object
Hour    object
HOEP    object
dtype: object
 Dataframe:2021.csv
2
Date    object
Hour    object
HOEP    object
dtype: object
 Dataframe:2022.csv
2
Date    object
Hour    object
HOEP    object
dtype: object
 Dataframe:2023.csv
2
Date    object
Hour    object
HOEP    object
dtype: object


In [462]:
# Handle data Type
HOEP_data_clean={key: value.astype({'HOEP':'float'}) for key,value in  HOEP_data_clean.items()}
# combining Date and Hour column into single column of datetime and changing the type to datetime:
#chalenge to do that to_datetime does not accept 24 as the value of Hour so have to deal with that

for key,value in HOEP_data_clean.items():
    value['Date']=pd.to_datetime(value['Date'])
    value['Hour'] = pd.to_numeric(value['Hour']) 
    value['Hour']=pd.to_timedelta(value['Hour']%24,unit='H')
    value['datetime']=value['Date']+value['Hour']
    value.set_index(value['datetime'], inplace=True)
    value.drop(['Hour','Date','datetime'],axis=1, inplace=True)
    
for key,value in HOEP_data_clean.items():
    print(f'dataframe {key} is:\n {value}')
#now our data is cleaned for each year and is ready to combine

dataframe 2018.csv is:
 2                     HOEP
datetime                  
2018-01-01 01:00:00  51.29
2018-01-01 02:00:00  43.59
2018-01-01 03:00:00  93.60
2018-01-01 04:00:00  54.78
2018-01-01 05:00:00  14.35
...                    ...
2018-12-31 20:00:00   5.62
2018-12-31 21:00:00   8.95
2018-12-31 22:00:00   5.81
2018-12-31 23:00:00   2.87
2018-12-31 00:00:00   0.00

[8760 rows x 1 columns]
dataframe 2019.csv is:
 2                    HOEP
datetime                 
2019-01-01 01:00:00  0.00
2019-01-01 02:00:00  0.00
2019-01-01 03:00:00 -0.11
2019-01-01 04:00:00 -1.84
2019-01-01 05:00:00 -0.47
...                   ...
2019-12-31 20:00:00  0.00
2019-12-31 21:00:00  0.00
2019-12-31 22:00:00 -0.05
2019-12-31 23:00:00 -0.04
2019-12-31 00:00:00 -0.02

[8760 rows x 1 columns]
dataframe 2020.csv is:
 2                     HOEP
datetime                  
2020-01-01 01:00:00   0.00
2020-01-01 02:00:00   0.00
2020-01-01 03:00:00   0.00
2020-01-01 04:00:00   0.00
2020-01-01 05:00:00   0.00


In [463]:
# Combine DataFrames along rows

final_df = pd.concat(HOEP_data_clean.values(), axis=0 )
print(final_df)

In [492]:
final_df.to_csv('final_df.csv')

3) EDA (Exploratory Data Analysis)